<a href="https://colab.research.google.com/github/angelinasi/Desafio_B2W/blob/master/teste_beam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Run and print a shell command.
def run(cmd):
  !{cmd}

# Install apache-beam.
run('pip install --quiet apache-beam')
run('pip install --quiet apache-beam[interactive]')

     |████████████████████████████████| 3.4MB 2.8MB/s 
     |████████████████████████████████| 235kB 30.0MB/s 
     |████████████████████████████████| 1.2MB 27.4MB/s 
     |████████████████████████████████| 59.2MB 70kB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
     |████████████████████████████████| 112kB 52.2MB/s 
ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: chainer 6.5.0 has requirement typing<=3.6.6, but you'll have typing 3.7.4.1 which is incompatible.
ERROR: chainer 6.5.0 has requirement typing-extensions<=3.6.6, but you'll have typing-extensions 3.7.4.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 61kB 3.4MB/s 
     |███████████████████████████████

In [0]:
from __future__ import absolute_import

import json
import datetime
import argparse

import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions


In [0]:
inputs_folder = 'input/page-views.json'
outputs_folder = 'output/abandoned-carts'

In [0]:
class JsonCoder(object):

  def encode(self, x):
    return json.dumps(x)

  def decode(self, x):
    return json.loads(x)

In [0]:
def filtrar(interacoes, carrinho, pagamento):
  interacao_carrinho = list(filter(lambda x: interacoes['customer'] in x, carrinho))
  interacao_pagamento = list(filter(lambda x: interacoes['customer'] in x, pagamento))

  data_interacao = datetime.datetime.strptime(interacoes['timestamp'],'%Y-%m-%d %H:%M:%S') 
  data_carrinho = [datetime.datetime.strptime(item[1],'%Y-%m-%d %H:%M:%S') for item in interacao_carrinho]
  data_pagamento = [datetime.datetime.strptime(item[1],'%Y-%m-%d %H:%M:%S') for item in interacao_pagamento]

  if(len(data_pagamento) > 0 and len(data_carrinho) > 0):
    intervalo = (data_pagamento[0] - data_carrinho[0]).total_seconds() / 60.0

  if((len(data_pagamento) <= 0 or intervalo > 10) and data_interacao == data_carrinho[0]):
      yield interacoes
  

In [0]:
def run(argv=None):
  
  parser = argparse.ArgumentParser()
  parser.add_argument('--input', required=True, help='Arquivo de entrada', default='input/page-views.json')
  parser.add_argument('--output', required=True, help='Arquivo de saída', default='output/abandoned-carts')
  known_args, pipeline_args = parser.parse_known_args(argv)
  pipeline_options = PipelineOptions(pipeline_args)

  with beam.Pipeline(options=PipelineOptions()) as pipeline:

    interacoes = pipeline | "lê arquivo de entrada" >> ReadFromText(known_args.input, coder=JsonCoder())

    data_ultima_interacao_carrinho = (
          interacoes
          | "filtrar as interações com o carrinho" >> beam.Filter(lambda elem: elem['page'] == 'basket')
          | "criar key = customer e value = timestamp --car" >> beam.Map(lambda elem: (elem['customer'], elem['timestamp']))
          | "agrupar pela data da ultima interação --car" >> beam.CombinePerKey(max)
    )  
    
    data_ultima_interacao_pagamento = (
          interacoes
          | "filtrar as interações com o pagamento" >> beam.Filter(lambda elem: elem['page'] == 'checkout')
          | "criar key = customer e value = timestamp --pag" >> beam.Map(lambda elem: (elem['customer'], elem['timestamp']))
          | "agrupar pela data da ultima interação --pag" >> beam.CombinePerKey(max)
    )  
  
    resultado = (
        interacoes
        | "verificar quais interações resultaram em abandono de carrinho" >>
          beam.FlatMap(filtrar, carrinho=beam.pvalue.AsList(data_ultima_interacao_carrinho),pagamento=beam.pvalue.AsList(data_ultima_interacao_pagamento))
        | "encoder para formato .json" >> beam.Map(json.dumps)
        | "cria arquivo de saída" >> WriteToText(known_args.output,'.json')
    )


In [0]:
if __name__ == '__main__':
  run()

usage: ipykernel_launcher.py [-h] --input INPUT --output OUTPUT
ipykernel_launcher.py: error: the following arguments are required: --input, --output


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  result.error_in_exec = value
